In [1]:
# Persistance in Langgraph referes to the ability to save and restore the state of the workflow over time.
# All intermidiate history as well is stored.
# Helps in fault tolerance as if workflow crashes we can resume from the last successfull node.

In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import google.generativeai as genai
from pydantic import BaseModel, Field

from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

/opt/anaconda3/envs/LangGraph-Tutorials/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import json
import re
from typing import Any

def extract_json(text: str) -> dict[str, Any]:
    """
    Extract and parse JSON object from text, ignoring markdown or natural language wrappers.
    """
    # Remove markdown formatting if present
    text = text.strip()
    if text.startswith("```json"):
        text = text[7:]
    if text.endswith("```"):
        text = text[:-3]

    # Find first JSON object using regex
    match = re.search(r'\{.*\}', text, re.DOTALL)
    if match:
        json_str = match.group(0)
        return json.loads(json_str)

    raise ValueError("No valid JSON object found in the response.")


# Extract schema structure as a string
def schema_prompt(schema_model: BaseModel) -> str:
    schema = schema_model.model_json_schema()
    # Only keep the "properties" part for a readable prompt
    return json.dumps(schema.get("properties", {}), indent=2)


In [4]:
import os
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# Configure the SDK
genai.configure(api_key=GEMINI_API_KEY)
# Create a model instance
model = genai.GenerativeModel("models/gemini-2.5-flash")

# Generate content
# prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n '
# response = model.generate_content(prompt)
# print(response.text)


In [5]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.generate_content(prompt)

    return {'joke': response.text}  # Only return the string

In [7]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.generate_content(prompt)

    return {'explanation': response.text}

In [8]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [9]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza have an identity crisis?\n\nBecause it was round, came in a square box, and you eat it in triangles!',
 'explanation': 'This joke plays on the literal meaning of an "identity crisis" applied to an inanimate object, using the different shapes associated with a pizza.\n\nHere\'s the breakdown:\n\n1.  **Identity Crisis:** An "identity crisis" is when someone feels confused or conflicted about who they are, their purpose, or their fundamental nature.\n2.  **The Pizza\'s "Conflicting Identities":**\n    *   **"It was round":** This is the pizza\'s original, baked shape. It\'s born round.\n    *   **"Came in a square box":** Despite being round, it\'s contained and transported in a box with a completely different shape – a square.\n    *   **"And you eat it in triangles!":** When it\'s served and consumed, it\'s typically cut into triangular slices, yet another distinct shape.\n\nThe humor comes from the absurdity of an object experiencing such 

In [10]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza have an identity crisis?\n\nBecause it was round, came in a square box, and you eat it in triangles!', 'explanation': 'This joke plays on the literal meaning of an "identity crisis" applied to an inanimate object, using the different shapes associated with a pizza.\n\nHere\'s the breakdown:\n\n1.  **Identity Crisis:** An "identity crisis" is when someone feels confused or conflicted about who they are, their purpose, or their fundamental nature.\n2.  **The Pizza\'s "Conflicting Identities":**\n    *   **"It was round":** This is the pizza\'s original, baked shape. It\'s born round.\n    *   **"Came in a square box":** Despite being round, it\'s contained and transported in a box with a completely different shape – a square.\n    *   **"And you eat it in triangles!":** When it\'s served and consumed, it\'s typically cut into triangular slices, yet another distinct shape.\n\nThe humor comes from the absurdity of an object

In [11]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza have an identity crisis?\n\nBecause it was round, came in a square box, and you eat it in triangles!', 'explanation': 'This joke plays on the literal meaning of an "identity crisis" applied to an inanimate object, using the different shapes associated with a pizza.\n\nHere\'s the breakdown:\n\n1.  **Identity Crisis:** An "identity crisis" is when someone feels confused or conflicted about who they are, their purpose, or their fundamental nature.\n2.  **The Pizza\'s "Conflicting Identities":**\n    *   **"It was round":** This is the pizza\'s original, baked shape. It\'s born round.\n    *   **"Came in a square box":** Despite being round, it\'s contained and transported in a box with a completely different shape – a square.\n    *   **"And you eat it in triangles!":** When it\'s served and consumed, it\'s typically cut into triangular slices, yet another distinct shape.\n\nThe humor comes from the absurdity of an objec

In [12]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': "Why did the pasta break up with the sauce?\n\nBecause he couldn't commit – he was always so **al dente**!",
 'explanation': 'This joke is a pun that plays on the literal meaning of the Italian culinary term **"al dente"** and its figurative association with a person\'s personality or relationship style.\n\nHere\'s the breakdown:\n\n1.  **Literal Meaning of "Al Dente":** In cooking, "al dente" (Italian for "to the tooth") describes pasta that is cooked until it\'s still firm when bitten, not soft or mushy. It has a slight resistance or "bite" to it.\n\n2.  **The Pun/Figurative Meaning:** The humor comes from applying this cooking term to a person\'s commitment level in a relationship.\n    *   If someone is "al dente" in a relationship, it implies they are still **firm**, **resistant**, and haven\'t fully **softened** or **yielded** to the commitment required in a serious relationship.\n    *   They are "not fully cooked" into the relationship, metaphoricall

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza have an identity crisis?\n\nBecause it was round, came in a square box, and you eat it in triangles!', 'explanation': 'This joke plays on the literal meaning of an "identity crisis" applied to an inanimate object, using the different shapes associated with a pizza.\n\nHere\'s the breakdown:\n\n1.  **Identity Crisis:** An "identity crisis" is when someone feels confused or conflicted about who they are, their purpose, or their fundamental nature.\n2.  **The Pizza\'s "Conflicting Identities":**\n    *   **"It was round":** This is the pizza\'s original, baked shape. It\'s born round.\n    *   **"Came in a square box":** Despite being round, it\'s contained and transported in a box with a completely different shape – a square.\n    *   **"And you eat it in triangles!":** When it\'s served and consumed, it\'s typically cut into triangular slices, yet another distinct shape.\n\nThe humor comes from the absurdity of an object

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza have an identity crisis?\n\nBecause it was round, came in a square box, and you eat it in triangles!', 'explanation': 'This joke plays on the literal meaning of an "identity crisis" applied to an inanimate object, using the different shapes associated with a pizza.\n\nHere\'s the breakdown:\n\n1.  **Identity Crisis:** An "identity crisis" is when someone feels confused or conflicted about who they are, their purpose, or their fundamental nature.\n2.  **The Pizza\'s "Conflicting Identities":**\n    *   **"It was round":** This is the pizza\'s original, baked shape. It\'s born round.\n    *   **"Came in a square box":** Despite being round, it\'s contained and transported in a box with a completely different shape – a square.\n    *   **"And you eat it in triangles!":** When it\'s served and consumed, it\'s typically cut into triangular slices, yet another distinct shape.\n\nThe humor comes from the absurdity of an objec

### Time Travel

In [17]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0706db-97f3-6468-8000-ee874a14a801"}})

StateSnapshot(values={'topic': 'pizza'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0706db-97f3-6468-8000-ee874a14a801'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-08-03T13:28:21.435297+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706db-97f2-6180-bfff-14e08a45e4b7'}}, tasks=(PregelTask(id='57b26d5a-5e7c-ded7-0ebe-35cc1ef59427', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pizza have an identity crisis?\n\nBecause it was round, came in a square box, and you eat it in triangles!'}),), interrupts=())

In [19]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0706db-97f3-6468-8000-ee874a14a801"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza get a job?\n\nBecause it **kneaded** the **dough**!',
 'explanation': 'This joke is a classic pun! It plays on the double meanings of two words: **"kneaded"** and **"dough."**\n\nLet\'s break it down:\n\n1.  **"Kneaded" (sounds like "needed")**:\n    *   **Literal meaning (for pizza):** When you make pizza from scratch, you have to **knead** the dough. This is a physical process of pressing and folding the mixture.\n    *   **Homophone meaning (for a person):** If someone "needed" a job, it means they **required** one, usually to earn money.\n\n2.  **"Dough"**:\n    *   **Literal meaning (for pizza):** This refers to the raw mixture of flour, water, yeast, etc., that you make the pizza *from*.\n    *   **Slang meaning (for a person):** "Dough" is a common, informal term for **money**.\n\n**Putting it together:**\n\nThe joke works because:\n\n*   **From a pizza\'s perspective:** A pizza conceptually "kneads" its own "dough" (the food kind) 

In [20]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it **kneaded** the **dough**!', 'explanation': 'This joke is a classic pun! It plays on the double meanings of two words: **"kneaded"** and **"dough."**\n\nLet\'s break it down:\n\n1.  **"Kneaded" (sounds like "needed")**:\n    *   **Literal meaning (for pizza):** When you make pizza from scratch, you have to **knead** the dough. This is a physical process of pressing and folding the mixture.\n    *   **Homophone meaning (for a person):** If someone "needed" a job, it means they **required** one, usually to earn money.\n\n2.  **"Dough"**:\n    *   **Literal meaning (for pizza):** This refers to the raw mixture of flour, water, yeast, etc., that you make the pizza *from*.\n    *   **Slang meaning (for a person):** "Dough" is a common, informal term for **money**.\n\n**Putting it together:**\n\nThe joke works because:\n\n*   **From a pizza\'s perspective:** A pizza conceptually "kneads" its own "dou

#### Updating State

In [21]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0706db-97f3-6468-8000-ee874a14a801", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0706e4-9459-66b4-8001-5865c8675bc0'}}

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706e4-9459-66b4-8001-5865c8675bc0'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-03T13:32:22.649588+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706db-97f3-6468-8000-ee874a14a801'}}, tasks=(PregelTask(id='a2ece96f-e80c-983e-3505-6b414fdb3c0e', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it **kneaded** the **dough**!', 'explanation': 'This joke is a classic pun! It plays on the double meanings of two words: **"kneaded"** and **"dough."**\n\nLet\'s break it down:\n\n1.  **"Kneaded" (sounds like "needed")**:\n    *   **Literal meaning (for pizza):** When you make pizza from 

In [25]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0706e4-9459-66b4-8001-5865c8675bc0"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa get an award?\nBecause it was outstanding in its *field*... of potatoes!',
 'explanation': 'This joke plays on a **pun**, specifically with the word "field" and its different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Outstanding in its *field*" (Idiomatic Meaning):**\n    *   This is a common English idiom meaning "exceptionally good" or "the best in a particular area, profession, or domain."\n    *   For example, "Dr. Smith is outstanding in her *field* of neuroscience."\n    *   When you first hear "Why did the samosa get an award? Because it was outstanding in its *field*...", your brain initially expects this meaning – that the samosa was an exceptionally good samosa, perhaps the best in the "culinary field."\n\n2.  **"Field of potatoes" (Literal/Pun Meaning):**\n    *   This is where the twist comes in. A samosa is a pastry *filled* with a mixture, often primarily potatoes.\n    *   The word "field" sounds very similar to "fille

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa get an award?\nBecause it was outstanding in its *field*... of potatoes!', 'explanation': 'This joke plays on a **pun**, specifically with the word "field" and its different meanings.\n\nHere\'s the breakdown:\n\n1.  **"Outstanding in its *field*" (Idiomatic Meaning):**\n    *   This is a common English idiom meaning "exceptionally good" or "the best in a particular area, profession, or domain."\n    *   For example, "Dr. Smith is outstanding in her *field* of neuroscience."\n    *   When you first hear "Why did the samosa get an award? Because it was outstanding in its *field*...", your brain initially expects this meaning – that the samosa was an exceptionally good samosa, perhaps the best in the "culinary field."\n\n2.  **"Field of potatoes" (Literal/Pun Meaning):**\n    *   This is where the twist comes in. A samosa is a pastry *filled* with a mixture, often primarily potatoes.\n    *   The word "field" sounds ve

### Fault Tolerance

In [33]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [34]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [35]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(10)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [36]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [37]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [38]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [39]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706f4-b716-60e8-8003-6128ec6c742f'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-03T13:39:35.788744+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706f4-b712-6f92-8002-ca9ee1dd161f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706f4-b712-6f92-8002-ca9ee1dd161f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-03T13:39:35.787459+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0706f4-3c85-62ca-8001-ca1be2212e51'}}, tasks=(PregelTask(id='536740a6-d701-186b-ec29-c291bdd9b48